In [3]:
import csv, os, re, shutil,json, sys
import pandas as pd 
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

### Find texts that contain sermons (DIV tag with attribute "TYPE"='sermon' or containing key terms in the title or subject headings)

In [2]:
from bs4 import BeautifulSoup, SoupStrainer
def is_sermon(filepath):
    # read the input XML file 
    with open(filepath,'r') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("DIV1")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    sermons = soup.findAll(attrs={"TYPE": "sermon"})
    if len(sermons) > 0:      
        return True
    return False

In [ ]:
metadataFolder = '/Users/amycweng/DH/ECBC-Data-2022/TCP metadata'
sermons = []
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID == "id": continue
        filepath = findTextTCP(tcpID)
        if is_sermon(filepath): 
            subject_headings = data['keywords'][idx].replace(" -- ","; ").replace("  "," ")
            sermons.append( {"id": tcpID, 
                            "estc":data['estc'][idx],
                            "stc":data['stc'][idx],
                            "title": data['title'][idx],
                            "authors": "; ".join(set(data['author'][idx].split("; "))),
                            "publisher": data['publisher'][idx],
                            "pubplace":data['pubplace'][idx],
                            "subject_headings":subject_headings,
                            "date":data['date'][idx]
                            }
            )
    print(csvFile)

In [ ]:
# store relevant metadata in a CSV file 
with open("../assets/sermons.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=sermons[0].keys())
    writer.writeheader()
    writer.writerows(sermons)

print(f"{len(sermons)} TCP XML files contain sermons.") 

4239 TCP XML files contain sermons.


In [2]:
sermons_metadata = pd.read_csv("../assets/sermons.csv")
sermons = sermons_metadata["id"]

In [69]:
# Find the texts that most likely contain sermons but do not have the proper div tag in the XML 

import os 
import pandas as pd 
metadataFolder = '/Users/amycweng/DH/ECBC-Data-2022/TCP metadata'
texts = {}
by_subj, by_title, by_both = [],[],[]
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        title = data['title'][idx]
        clean_title = title.lower().replace("'",'')
        subject_headings = data['keywords'][idx]
        unique_subjects = " -- ".join(set(subject_headings.split(" -- ")))
        subject_hit, title_hit = False, False
        if re.search('sermon', subject_headings.lower()): 
            subject_hit = True 
        if re.search(r'sermon|preached|preacht|preachd', clean_title): 
            title_hit = True
        if subject_hit and title_hit: 
            by_both.append(tcpID)
        elif subject_hit: 
            by_subj.append(tcpID)
        elif title_hit: 
            by_title.append(tcpID)
        if subject_hit or title_hit:  
            texts[tcpID] =  {"id": tcpID, 
                            "estc":data['estc'][idx],
                            "stc":data['stc'][idx],
                            "title": data['title'][idx],
                            "authors": "; ".join(set(data['author'][idx].split("; "))),
                            "publisher": data['publisher'][idx],
                            "pubplace":data['pubplace'][idx],
                            "subject_headings":unique_subjects,
                            "date":data['date'][idx]
                            }
print(len(texts))

5839


In [10]:
new = []
sermons = {k:None for k in sermons}
import os 
import pandas as pd 
metadataFolder = '/Users/amycweng/DH/ECBC-Data-2022/TCP metadata'
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID in sermons: 
            title = data['title'][idx]
            clean_title = title.lower().replace("'",'')
            subject_headings = data['keywords'][idx]
            unique_subjects = " -- ".join(set(subject_headings.split(" -- ")))  
            new.append({"id": tcpID, 
                        "estc":data['estc'][idx],
                        "stc":data['stc'][idx],
                        "title": data['title'][idx],
                        "authors": "; ".join(set(data['author'][idx].split("; "))),
                        "publisher": data['publisher'][idx],
                        "pubplace":data['pubplace'][idx],
                        "subject_headings":unique_subjects,
                        "date":data['date'][idx]
                        })
print(len(new))

4235


In [11]:
with open("../assets/sermons.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=new[0].keys())
    writer.writeheader()
    writer.writerows(new)

In [53]:
sermons = {tcpID:0 for tcpID in sermons}

In [70]:
missing_b, missing_s, missing_t = [],[],[]
missing = {0:[], 1:[], 2:[]}
for idx, collection in enumerate([by_both, by_subj, by_title]): 
    for tcpID in collection: 
        if tcpID not in sermons: missing[idx].append(tcpID)

In [71]:
print(len(missing[0]),len(missing[1]),len(missing[2]))

1081 313 393


In [56]:
exclude = ['B01388', 'A89416', 'B28836', 'B26622', 'B12473', 'B03839', 'B06138', 'B09463', 'A89104', 'A83012', 'A81906', 'A84063', 'A81606', 'A88596', 'A81417', 'A85341', 'A90702', 'A96864', 'A93332', 'A90476', 'A95939', 'A79568', 'A73832', 'A77100', 'A63877', 'A62992', 'A67411', 'A60864', 'A64639', 'A64197', 'A61683', 'A48191', 'A43685', 'A43806', 'A46883', 'A40538', 'A41496', 'A45149', 'A48968', 'A56791', 'A55289', 'A54939', 'A57258', 'A56278', 'A58892', 'A16497', 'A18267', 'A03696', 'A06013', 'A31039', 'A60568', 'A64394', 'A68546', 'A65419', 'A77638', 'A78013', 'A42577', 'A47973', 'A42539', 'A45574', 'A00164', 'A00156', 'A09418', 'A11848', 'A18019', 'A13299', 'A14927', 'A30903', 'A36190', 'A31459', 'A26859', 'A26426', 'A26065']

In [72]:
missing_texts = []
title_only = []
for idx, collection in missing.items(): 
    ''' 
    Exclude the missing TCP ids that only have key terms in the title but not the subject headings.
    These are the ones that are most likely to be false positives. 
    '''
    if idx == 2: 
        for tcpID in collection: 
            title_only.append(texts[tcpID]) 
    else: 
        for tcpID in collection: 
            if tcpID in exclude: continue
            missing_texts.append(texts[tcpID])
len(missing_texts), len(title_only)

(1359, 393)

In [ ]:
for t in texts.values(): 
    t = t["id"]
    path = findTextTCP(t)
    shutil.copy(path,'/Users/amycweng/DH/sermonsTCP')

In [73]:
unknown = []
for t in title_only: 
    tcpID = t["id"]
    title = t["title"]
    subjects = t["subject_headings"]
    if tcpID in sermons: continue
    if tcpID in exclude: 
        unknown.append(texts[tcpID])
        continue
    if re.search("answer|aunswer|ansvvere",title):
        if not re.search("substance of", title): 
            unknown.append(texts[tcpID])
        else: 
            missing_texts.append(texts[tcpID])
    elif re.search('sermon',title): 
        missing_texts.append(texts[tcpID])
    else: 
        unknown.append(texts[tcpID])

In [74]:
print(len(missing_texts), len(unknown))

1587 165


In [75]:
with open("../assets/sermons_missing.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=missing_texts[0].keys())
    writer.writeheader()
    writer.writerows(missing_texts)

In [76]:
with open("../assets/sermons_unknown.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=unknown[0].keys())
    writer.writeheader()
    writer.writerows(unknown)

# Map section names to indices 

In [3]:
import os,re
import pandas as pd 
from tqdm import tqdm 

sermons_metadata = pd.read_csv("../assets/sermons.csv")
sermons = list(sermons_metadata["id"])
missing_sermons = pd.read_csv("../assets/sermons_missing.csv")
sermons.extend(list(missing_sermons["id"]))
sermons = {s:None for s in sermons}
num_oral = 0 
sections = [] # tcpID to section to index 
for fp in tqdm(os.listdir("../assets/plain_all")): 
    if fp == ".DS_Store": continue 
    with open(f"../assets/plain_all/{fp}","r") as file: 
        text = file.read()
    tcpID = fp.split(".")[0]
    if tcpID not in sermons: continue 
    s = re.findall(r"SECTION(\d+):(\w+)\b",text)
    for idx, name in s: 
        if re.search(r"sermon|speech|oratio|homil|eulog|lecture|encomi|exhortation|memorial|consolatio",name):
            num_oral += 1  
        sections.append({"tcpID":tcpID, "section_idx":idx,"section_name":name})
sections = pd.DataFrame(sections)
sections.to_csv('/Users/amycweng/DH/SERMONS_APP/db/data/sections.csv',header=False,index=False)
print(num_oral)
sections

100%|██████████| 5862/5862 [00:03<00:00, 1927.92it/s]


10767


,tcpID,section_idx,section_name
0,A57739,0,title_page
1,A57739,1,to_the_reader
2,A57739,2,sermon
3,A67822,0,title_page
4,A67822,1,translator_to_author
...,...,...,...
35126,A41142,2,letter
35127,A41142,3,to_the_reader
35128,A41142,4,table_of_contents
35129,A41142,5,sermon
